In [1]:
import numpy as np
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()

In [3]:
def extract_text(url):
    
    driver.get(url)
    time.sleep(5)
    
    
    content = driver.page_source
    soup = BeautifulSoup(content)
   
    part_no = soup.find('span',attrs={'class':'part-info-spec__value'})
    try:
        part_no=part_no.text[1:-1]
    except:
        driver.get(url)
        time.sleep(10)
        content = driver.page_source
        soup = BeautifulSoup(content)
   
        part_no = soup.find('span',attrs={'class':'part-info-spec__value'})
        part_no=part_no.text[1:-1]
    
    sp=part_no.split("-")
    if len(sp)!=2:
        part_no=str(part_no[:5])+"-"+part_no[5:]
    
    #brand=soup.find('a',href=True, attrs={'class':'part-info-top__brand'})
    #try:
        #if brand.text:
            #brand=brand.text[1:-1]
        #else:
            #brand=brand.get("href").split("-")[-1][:-1]
    #except:
        #print("error in brand")
    #print(brand)
    
    brand= "Hyundai/Kia"
    
    desc=soup.find('h2', attrs={'class':'part-info-heading'})
    desc=desc.text[1:-1]
    
    lol=[]
    for a in soup.findAll('a',href=True, attrs={'class':'ng-star-inserted'}):
        lol.append(a)
    #category = lol[3].text[1:-1]
    category = "Exhaust System"
    sub_category=lol[4].text[1:-1]
    
    price=soup.find('span', attrs={'class':'part-info-price__mrp'})
    try:
        price=price.text[6:]
    except:
        print("no price")
        return
    price=price.split(",")
    pric=""
    for i in price:
        pric=pric+i
    price=int(pric)
    
    com_li=[]
    for a in soup.findAll('li', attrs={'class':'compatibility-nav__list__item'}):
        com_li.append(a.text[1:-1])
    if len(com_li)==0:
        print("no data")
        return
    
    make1= "Hyundai"
    make2="KIA"
    
    make_len_kia=0
    if len(com_li)==1:
        make1_name = com_li[0].split(" ")[0]
        if make1_name!="HYUNDAI":
            make1="KIA"
        make_len = int(com_li[0].split(" ")[1][1:-1])

    if len(com_li)==2:
        print(url)
        make_len = int(com_li[0].split(" ")[1][1:-1])
        make_len_kia = int(com_li[1].split(" ")[1][1:-1])
    
    
    if make_len>10:
        WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "span.btn.btn-sm"))).click()
        content = driver.page_source
        soup = BeautifulSoup(content)
        
    li_model=[]
    for a in soup.findAll('div', attrs={"data-head-title":"Model"}):
        li_model.append(a.text[1:-1])
    
    li_year=[]
    for a in soup.findAll('div', attrs={"data-head-title":"Year"}):
        li_year.append(a.text[1:-1])
    
    li_engine=[]
    for a in soup.findAll('div', attrs={"data-head-title":"Engine"}):
        li_engine.append(a.text[1:-1])
    
    li_power=[]
    for a in soup.findAll('div', attrs={"data-head-title":"Power (hp)"}):
        li_power.append(a.text[1:-1])
    
    li_fuel=[]
    for a in soup.findAll('div', attrs={"data-head-title":"Fuel type"}):
        li_fuel.append(a.text[1:-1])
    
    li_engine_type=[]
    for a in soup.findAll('div', attrs={"data-head-title":"Engine type"}):
        li_engine_type.append(a.text[1:-1])
    
    df=[]
    for i in range(len(li_model)):
        df.append([part_no,brand,"",desc,price,category,sub_category,"",make1,li_model[i],li_year[i],li_engine[i],li_power[i],li_fuel[i],li_engine_type[i]])
        
    if make_len_kia>0:
        for i in range(make_len_kia):
            df.append([part_no,brand,"",desc,price,category,sub_category,"",make2])
        
    return df

In [4]:
def extract_url(url):
    
    driver.get(url)
    time.sleep(5)
    
    content = driver.page_source
    soup = BeautifulSoup(content)
    
    li=[]
    for a in  soup.findAll('a',href=True,attrs={'class':'product-item-list__images'}):
        a=a.get("href")
        li.append("https://boodmo.com"+a)
    return li

In [15]:
uri=["https://boodmo.com/brands/1254-hyundaikia/4557-turbocharger/",
     "https://boodmo.com/brands/1254-hyundaikia/4559-turbocharger_gasket/",
     "https://boodmo.com/brands/1254-hyundaikia/4562-turbocharger_hose/",
     "https://boodmo.com/brands/1254-hyundaikia/4562-turbocharger_hose/page-2/",
     "https://boodmo.com/brands/1254-hyundaikia/4562-turbocharger_hose/page-3/",
     "https://boodmo.com/brands/1254-hyundaikia/4562-turbocharger_hose/page-4/",
     "https://boodmo.com/brands/1254-hyundaikia/4562-turbocharger_hose/page-5/",
     "https://boodmo.com/brands/1254-hyundaikia/4562-turbocharger_hose/page-6/"
    ]

In [16]:
li=[]
for i in uri:
    a=extract_url(i)
    for j in a:
        li.append(j)

In [17]:
len(li)

200

In [18]:
df1 = pd.DataFrame()
df1

""


In [19]:
for i in range(len(li)):
    l=extract_text(li[i])
    df2=pd.DataFrame(l)
    df1=pd.concat([df1, df2])
    print(i)

0
no data
1
https://boodmo.com/catalog/part-turbocharger-58208382/
2
3
4
5
https://boodmo.com/catalog/part-turbocharger-58201240/
6
7
8
9
https://boodmo.com/catalog/part-turbocharger-58200798/
10
11
12
13
14
15
16
no data
17
18
19
no price
20
no price
21
no price
22
no price
23
no price
24
25
26
27
28
https://boodmo.com/catalog/part-gasket_turbocharger_outlet-6809667/
29
30
https://boodmo.com/catalog/part-gasket_turbocharger_outlet-6803503/
31
32
33
34
35
36
37
https://boodmo.com/catalog/part-gasket_t_c_oil_drain_b-4563114/
38
39
40
41
42
43
44
45
46
47
https://boodmo.com/catalog/part-packing_cargo-6678903/
48
49
50
51
52
https://boodmo.com/catalog/part-hose_a-6806340/
53
54
https://boodmo.com/catalog/part-hose_i_c_outlet-58206241/
55
56
57
58
59
60
61
62
63
64
https://boodmo.com/catalog/part-hose_i_cooler_inlet-58206242/
65
66
67
68
69
70
71
https://boodmo.com/catalog/part-hose_b-58200073/
72
73
74
75
https://boodmo.com/catalog/part-hose_a-57674380/
76
https://boodmo.com/catalog/part-

In [20]:
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,28201-2A070,Hyundai/Kia,,TURBOCHARGER,15731,Exhaust System,Air Supply,,Hyundai,AURA 1.2L,01.2020 - 01.2023,1.2 L,73.5 h.p.,Diesel,ECO TORQ
1,28201-2A070,Hyundai/Kia,,TURBOCHARGER,15731,Exhaust System,Air Supply,,Hyundai,AURA 1.2L AT,01.2020 - 01.2023,1.2 L,73.5 h.p.,Diesel,ECO TORQ
2,28201-2A070,Hyundai/Kia,,TURBOCHARGER,15731,Exhaust System,Air Supply,,Hyundai,GRAND I10 F/L 1.2L,12.2016 - 11.2019,1.2 L,75 h.p.,Diesel,CRDi
3,28201-2A070,Hyundai/Kia,,TURBOCHARGER,15731,Exhaust System,Air Supply,,Hyundai,GRAND I10 NIOS 1.2L,08.2019 - 12.2022,1.2 L,74 h.p.,Diesel,CRDi
4,28201-2A070,Hyundai/Kia,,TURBOCHARGER,15731,Exhaust System,Air Supply,,Hyundai,GRAND I10 NIOS 1.2L AT,08.2019 - 12.2022,1.2 L,74 h.p.,Diesel,CRDi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,28274-2A490,Hyundai/Kia,,HOSE D,21384,Exhaust System,Air Supply,,Hyundai,CRETA 1ST GEN F/L 1.4L S,05.2018 - 03.2020,1.4 L,88 h.p.,Diesel,CRDi
5,28274-2A490,Hyundai/Kia,,HOSE D,21384,Exhaust System,Air Supply,,Hyundai,CRETA 1ST GEN F/L 1.6L SX(O),05.2018 - 03.2020,1.6 L,126 h.p.,Diesel,CRDi-VGT
6,28274-2A490,Hyundai/Kia,,HOSE D,21384,Exhaust System,Air Supply,,Hyundai,CRETA 1ST GEN F/L 1.6L SX,05.2018 - 03.2020,1.6 L,126 h.p.,Diesel,CRDi-VGT
0,28250-2A401,Hyundai/Kia,,PIPE ASSY-INTERCOOLER INLET,21622,Exhaust System,Air Supply,,Hyundai,VERNA 3RD GEN 1.5L,09.2006 - 09.2009,1.5 L,110 h.p.,Diesel,D4FA


In [21]:
len(pd.unique(df1[0]))

170

In [23]:
df1.to_excel("my.xlsx")